# Proyecto Imágenes Laboratorio Inteligencia

### Profesor: Claudio Pérez
### Auxiliar: Juan Pérez
### Integrantes: Matías Osses - Ignacio - Alejandro

El objetivo de este proyecto final es desarrollar e implementar un algoritmo de búsqueda de imágenes similares basado en su contenido (Content Based Image Retrieval, CBIR) utilizando distintos métodos de extracción de características. Para el desarrollo de este proyecto debe descargar la base de datos INRIA Holidays dataset. Debe descargar los archivos jpg1.tar.gz y jpg2.tar.gz que contienen imágenes de consulta para 500 clases distintas y las imágenes que deberán ser almacenadas en el buscador, y que están asociadas a esas 500 clases. La figura 1 muestra las principales etapas de un sistema CBIR.


## Importación de librerías

In [7]:
import os
import copy

import cv2
import numpy as np
import matplotlib.pyplot as plt


## Cálculo de vector de características

In [49]:
class Imagen():
    
    def __init__(self, path):
        self.nombre = path.split('\\')[-1]
        self.img = cv2.imread(path)
        self.img = cv2.cvtColor(self.img, cv2.COLOR_BGR2HSV)
        self.histogramas = dict()
        
    def division(self, n=3):
        nombre = '%sx%s'%(n,n)
        rows, cols, depth = self.img.shape
        histH = []
        histS = []
        histV = []
        for y in range(0, rows, n):
            for x in range(0, cols, n):
                celda = self.img[y:y+n, x:x+n, 0]
                histH.append(np.histogram(celda, bins=np.linspace(0,255,8)))
                celda = self.img[y:y+n, x:x+n, 1]
                histS.append(np.histogram(celda, bins=np.linspace(0,255,12)))
                celda = self.img[y:y+n, x:x+n, 2]
                histV.append(np.histogram(celda, bins=np.linspace(0,255,3))) 
        
        self.histogramas[nombre+'_H'] = np.array(histH)
        self.histogramas[nombre+'_S'] = np.array(histS)
        self.histogramas[nombre+'_V'] = np.array(histV)
        
    def calcularVectorCaracteristicas(self):
        self.division(3)
        self.division(4)
        self.division(6)
        

In [50]:
path = os.getcwd() + '\jpg'
img = path + '\\130000.jpg'

a = Imagen(img)
a.calcularVectorCaracteristicas()


1920000
1920000
1920000


In [51]:
print(a.histogramas['3x3_H'][1,2])

IndexError: index 2 is out of bounds for axis 1 with size 2

In [46]:
print(np.linspace(0,255,8))

[  0.          36.42857143  72.85714286 109.28571429 145.71428571
 182.14285714 218.57142857 255.        ]


In [ ]:
print()